In [1]:
# 重新加载模块
%load_ext autoreload
%autoreload 2
from utils.DataPreProcessor_0605 import DataPreProcessor 

sensi_names = ["sex"]
processor = DataPreProcessor(
    data_path="../input/adult.csv", 
    sensitive_names=sensi_names,
    label="income",
    label_mapper={"<=50K":0, ">50K":1}
)


{(0.703665851855809,): ('Male',), (-1.4211290733558493,): ('Female',)}


[22:24:50] INFO     数据集处理完毕， 遍历 xxx.grouped                                   ]8;id=781036;file://d:\Dropbox\Projects\joyful-pandas\exp\utils\DataPreProcessor_0604.py\DataPreProcessor_0604.py]8;;\:]8;id=274920;file://d:\Dropbox\Projects\joyful-pandas\exp\utils\DataPreProcessor_0604.py#44\44]8;;\
                    查看数据，每个单元是Group(scale_name, origin_name , X, y)                                      

In [70]:
group = processor.grouped[0]

In [126]:

X_train = processor.X_train_label_scale
y_train = processor.y_train
X_test = processor.X_test_label_scale
y_test = processor.y_test
sensitive_features = processor.X_test[sensi_names]

X_train.shape, y_train.shape, X_test.shape, y_test.shape, 

((22792, 14), (22792,), (9769, 14), (9769,))

## 模型与预测

In [5]:
import xgboost
model = xgboost.XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
# 敏感属性

In [135]:
import pandas as pd
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    # r2_score, # 决定系数，通常和回归问题相关
    # roc_curve, # 返回的数组，暂时不考虑
    # classification_report, # 一个报告，用处不大
)

from fairlearn.metrics import (
    demographic_parity_difference,
    demographic_parity_ratio,
    equalized_odds_difference,
    equalized_odds_ratio,
)


class FairMetric:

    def __init__(self, name, y_true, y_pred, sensitive_features: pd.Series = None):
        self.y_true = y_true
        self.name = name
        self.y_pred = y_pred
        self.sensitive_features = sensitive_features
        self.performance_metrics = pd.Series(name="Performance" + name)
        self.fairness_metrics = pd.Series(name="Fair" + name)
        self.handle_metrics()

    def eval_performance_metrics(self):
        """评估性能指标"""
        metrics = self.performance_metrics
        y_true, y_pred = self.y_true, self.y_pred
        metrics["准确度(1)"] = accuracy_score(y_true, y_pred)
        metrics["精确度(1)"] = precision_score(y_true, y_pred)
        metrics["召回率(1)"] = recall_score(y_true, y_pred)
        metrics["f1分数(1)"] = f1_score(y_true, y_pred)
        # s['回归系数(1)'] = r2_score(y_true, y_pred)
        metrics["AUC分数(1)"] = roc_auc_score(y_true, y_pred)

    def eval_fairness_metrics(self):
        """评估公平性指标"""
        if self.sensitive_features is None:
            raise ValueError("敏感属性未赋值")
        sf = self.sensitive_features
        metrics = self.fairness_metrics
        y_true, y_pred = self.y_true, self.y_pred
        metrics["DP差异(0)"] = demographic_parity_difference(
            y_true, y_pred, sensitive_features=sf
        )
        metrics["DP比率(1)"] = demographic_parity_ratio(
            y_true, y_pred, sensitive_features=sf
        )
        metrics["EO差异(0)"] = equalized_odds_difference(
            y_true, y_pred, sensitive_features=sf
        )
        metrics["EO比率(1)"] = equalized_odds_ratio(
            y_true, y_pred, sensitive_features=sf
        )

    def handle_metrics(self):
        """评估性能和公平性指标"""
        self.eval_performance_metrics()
        self.eval_fairness_metrics()
        self.metrics = pd.concat([self.performance_metrics, self.fairness_metrics])
        self.metrics.name = self.name

In [137]:
fm = FairMetric("XGBClassifier-baseline", y_test, y_pred, sensitive_features=sensitive_features)
fm.metrics

准确度(1)      0.871635
精确度(1)      0.772044
召回率(1)      0.658547
f1分数(1)     0.710793
AUC分数(1)    0.798650
DP差异(0)     0.186382
DP比率(1)     0.298818
EO差异(0)     0.069172
EO比率(1)     0.215589
Name: XGBClassifier-baseline, dtype: float64